In [ ]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

# 데이터호출

In [5]:
## 키워드 작업한 데이터 불러오기
import pandas as pd
import torch

df_fraud = pd.read_csv('data/df_fraud_keyword.csv', index_col=False)

In [10]:
## DB에 저장한 키워드작업된 데이터 가져오기(사기기)
import mysql.connector
import pandas as pd
from db_info import user, password, host, port, database

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_keyword")

result = cursor.fetchall()

df_injury = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [11]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  3821 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
dtypes: int64(2), object(9)
memory usage: 360.1+ KB


In [5]:
df_injury.drop(columns=['판례정보일련번호','사건번호.1'], inplace=True)
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4187 entries, 0 to 4186
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4187 non-null   int64 
 1   사건명            4186 non-null   object
 2   사건번호           4187 non-null   object
 3   선고일자           4187 non-null   object
 4   법원명            4187 non-null   object
 5   사건종류명          4187 non-null   object
 6   판례상세링크         4187 non-null   object
 7   선고             4187 non-null   object
 8   판결유형           4187 non-null   object
 9   판시사항           3415 non-null   object
 10  판결요지           3026 non-null   object
 11  참조조문           3390 non-null   object
 12  참조판례           1924 non-null   object
 13  판례내용           4187 non-null   object
 14  판례내용_상단        4187 non-null   object
 15  판례내용_이유        4187 non-null   object
 16  판례내용_이유_전처리    4187 non-null   object
 17  판례내용_이유_불용어제거  4187 non-null   object
 18  클러스터           4187 non-null

In [57]:
df_fraud[df_fraud['사건명'].isnull()]

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,선고,판결유형,...,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드
3478,71592,None,2001노3042,2002.04.12,서울고등법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,71592,선고,판결：상고기각,...,[1] 특정범죄가중처벌등에관한법률 제5조의2 제4항에서 영리의 목적...,"\n[1]\n\n 형법 제288조 제1항\n,\n\n ...",None,【피고인】 피고인【항소인】 피고인【변호인】 변호사 이수형 외 1인【원심판결】 ...,【피고인】 피고인【항소인】 피고인【변호인】 변호사 이수형 외 1인【원심판결】 ...,【이유】1. 항소이유의 요지가. 사실오인피고인은 무허가 직업소개소를 운영하면서 원심...,{'이유': '1. 항소이유의 요지가. 사실오인피고인은 무허가 직업소개소를 운영하면...,사실오인 무허가 직업 소개 운영 데리 도록 망하여 유인 의사 과장 하였을 이미 다...,7,"['지급', '계약', '사기죄', '채권', '신청', '회사', '공사', '의..."
3793,105119,None,86다카1147,1989.04.25,대법원,민사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,105119,선고,판결,...,가. 계약이 합의해제된 경우에는 그 해제시에 당사자 일방이 상대방에게 손해배상을 하...,가.\n \n 민법 제551조 / 나. \n 제664조<br/>,가. \n \n대법원 1960.10.6. 선고 4293민상275 판결 ...,【원고(반소피고)】 상고인 곽태훈【피고(반소원고)】 피상고인 덕화공영주식...,【원고(반소피고)】 상고인 곽태훈【피고(반소원고)】 피상고인 덕화공영주식...,"【이 유】 1. 상고이유 제1점을 본다. 원심판결 이유에 의하면, 원심은 ...","{'이유': '1. 상고이유 제1점을 본다. 원심판결 이유에 의하면, 원심은 원고(...",본다 손해배상 1983.5 덕화 주식회사 소원 회사 와의 소유 대지 건축 지하 지상...,2,"['회사', '주식', '주식회사', '자금', '대출', '거래', '투자', '..."


In [28]:
## DB에 저장한 키워드작업된 데이터 가져오기
import mysql.connector
import pandas as pd
from db_info import user, password, host, port, database

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_keyword")

result = cursor.fetchall()

df_injury = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [29]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  4184 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
dtypes: int64(2), object(9)
memory usage: 360.1+ KB


In [30]:
## DB에 저장한 키워드작업된 데이터 가져오기
import mysql.connector
import pandas as pd
from db_info import user, password, host, port, database

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_list")

result = cursor.fetchall()

injury_list = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [31]:
injury_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  4189 non-null   int64 
 1   사건명     4188 non-null   object
 2   사건번호    4189 non-null   object
 3   선고일자    4189 non-null   object
 4   법원명     4189 non-null   object
 5   사건종류명   4189 non-null   object
 6   판례상세링크  4189 non-null   object
dtypes: int64(1), object(6)
memory usage: 229.2+ KB


In [32]:
merged_df = df_injury.merge(injury_list[['사건번호','사건명']], on='사건번호', how='left')

df_injury['사건명'] = merged_df['사건명']

In [33]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  4184 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
 11  사건명            4188 non-null   object
dtypes: int64(2), object(10)
memory usage: 392.8+ KB


# 승민님 BERT모델(판시사항)

In [1]:
print(df_fraud['판시사항'].iloc[10])


NameError: name 'df_fraud' is not defined

In [ ]:
df_fraud['모델용_문장'] = "[CLS]" + df_fraud['판시사항'] + "[SEP]"

In [ ]:
df_fraud['모델용_문장'].iloc[1]

'[CLS] 배심원이 참여하는 형사재판, 즉 국민참여재판을 거쳐 제1심법원이 배심원의 만장일치 무죄평결을 받아들여 피고인에 대하여 무죄판결을 선고한 경우, ‘증거의 취사 및 사실의 인정’에 관한 제1심법원의 판단은 한층 더 존중될 필요가 있는지 여부(적극) 및 이때 제1심법원의 무죄판결에 대한 항소심에서의 추가적이거나 새로운 증거조사의 범위[SEP]'

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from kobert_transformers import get_tokenizer

bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

# Define your model
model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

for i in range(10):
    sentences = []
    sentences.append(df_fraud['모델용_문장'].iloc[i])


# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# Print the tensor
print(tokenid_tensor)

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

c:\Users\user\anaconda3\envs\nof\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tensor([[   2, 3893, 5478, 6335, 4128,  150, 7253, 4128,   93, 7846, 6903, 4092,
         7828, 4071, 7202, 3480, 5384, 7096, 4596, 5898, 3162, 2682, 3893,  517,
         7656, 5781, 7088, 4955, 2587, 1528, 6006,  833, 2322, 4128,  225, 7253,
         4128,  116, 7925, 6896, 1835, 4466, 5889, 7318, 3310,   18, 7202, 5543,
           40,    3]])
torch.Size([1, 50, 768])
tensor([[[ 0.0527,  0.2446,  0.1086,  ..., -0.5052, -0.5448,  0.1624],
         [ 0.6586,  0.0192,  0.1596,  ...,  0.0030, -0.1197,  0.2213],
         [ 0.4585, -0.0808,  0.4904,  ...,  0.1427, -0.2438, -0.2900],
         ...,
         [ 0.6087, -0.0853,  0.3526,  ..., -0.3582, -0.3065, -0.0323],
         [ 0.1492,  0.0111,  0.4568,  ..., -0.2438,  0.0845, -0.7004],
         [ 0.0525,  0.2449,  0.1089,  ..., -0.5057, -0.5442,  0.1627]]],
       grad_fn=<NativeLayerNormBackward0>)


# Bert코드

In [34]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#KoBERT모델, 토크나이저 생성
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)
    with torch.no_grad():
        # outputs = model(**inputs)
        outputs = model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용, 키워드, 판결요지, 판시사항 임베딩 생성
df_injury['판례내용이유임베딩'] = df_injury['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['키워드임베딩'] = df_injury['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['판결요지임베딩'] = df_injury['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['판시사항임베딩'] = df_injury['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_injury.to_csv('data/df_injury_embedding.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#KoBERT모델, 토크나이저 생성
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)
    with torch.no_grad():
        # outputs = model(**inputs)
        outputs = model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용, 키워드, 판결요지, 판시사항 임베딩 생성
df_fraud['판례내용이유임베딩'] = df_fraud['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['키워드임베딩'] = df_fraud['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['판결요지임베딩'] = df_fraud['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['판시사항임베딩'] = df_fraud['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_fraud.to_csv('data/df_fraud_embedding.csv', index=False)

In [61]:
df_fraud.drop(columns=['선고일자','법원명','사건종류명','판례상세링크','판례정보일련번호','선고'], inplace=True)

In [4]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  3821 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
 11  판례내용이유임베딩      4184 non-null   object
 12  키워드임베딩         4189 non-null   object
 13  판결요지임베딩        3028 non-null   object
 14  판시사항임베딩        3417 non-null   object
dtypes: int64(2), object(13)
memory usage: 491.0+ KB


In [62]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4178 entries, 0 to 4177
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4178 non-null   int64 
 1   사건명            4176 non-null   object
 2   사건번호           4178 non-null   object
 3   판결유형           4178 non-null   object
 4   판시사항           3025 non-null   object
 5   판결요지           2479 non-null   object
 6   참조조문           2998 non-null   object
 7   참조판례           1805 non-null   object
 8   판례내용           4178 non-null   object
 9   판례내용_상단        4178 non-null   object
 10  판례내용_이유        4178 non-null   object
 11  판례내용_이유_전처리    4178 non-null   object
 12  판례내용_이유_불용어제거  4178 non-null   object
 13  클러스터           4178 non-null   int64 
 14  키워드            4178 non-null   object
 15  판례내용이유임베딩      4178 non-null   object
 16  키워드임베딩         4178 non-null   object
 17  판결요지임베딩        2479 non-null   object
 18  판시사항임베딩        3025 non-null

In [ ]:
### 진형코드
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    if pd.notnull(sentence):
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()
    return None

# 클러스터별 판례 내용과 키워드 임베딩 생성
for cluster_num in range(1, 9):  # 클러스터 1부터 8까지
    cluster_cases = df_inheritance[df_inheritance['클러스터'] == cluster_num]
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판례내용이유임베딩'] = \
        cluster_cases['판례내용_이유'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '키워드임베딩'] = \
        cluster_cases['키워드'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판결요지임베딩'] = \
        cluster_cases['판결요지'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판시사항임베딩'] = \
        cluster_cases['판시사항'].apply(encode_sentence)

# CSV 파일로 저장
df_inheritance.to_csv('df_inheritance_kobertmodel2.csv', index=False)

# 유사도 계산

In [35]:
import ast
df_injury = pd.read_csv('data/df_injury_embedding.csv', index_col=False)

# 다시 리스트를 NumPy 배열로 변환
df_injury['판례내용이유임베딩'] = df_injury['판례내용이유임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_injury['키워드임베딩'] = df_injury['키워드임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_injury['판결요지임베딩'] = df_injury['판결요지임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_injury['판시사항임베딩'] = df_injury['판시사항임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)

In [36]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  4184 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
 11  사건명            4188 non-null   object
 12  판례내용이유임베딩      4184 non-null   object
 13  키워드임베딩         4189 non-null   object
 14  판결요지임베딩        3028 non-null   object
 15  판시사항임베딩        3417 non-null   object
dtypes: int64(2), object(14)
memory usage: 523.8+ KB


In [37]:
df_injury['판례내용이유임베딩'].iloc[0]

array([[ 1.09782562e-01, -2.86987662e-01,  6.38796270e-01,
        -3.11709237e+00,  1.48222923e-01,  3.20461363e-01,
         1.69980347e-01,  1.59377038e-01, -2.80434996e-01,
        -2.74808586e-01, -2.18382433e-01,  3.88051644e-02,
        -7.76524544e-02,  3.70469391e-01,  7.92211175e-01,
        -3.30990590e-02,  7.45082945e-02,  2.26210415e-01,
         4.97903600e-02, -3.07749033e-01, -4.67122734e-01,
        -9.32853818e-02, -1.44847378e-01,  1.82107747e-01,
        -1.09147832e-01, -1.59866475e-02,  1.72302067e-01,
        -1.93120167e-01, -2.18056329e-02,  7.01031566e-01,
         5.00526652e-02, -3.53494287e-01,  7.59904608e-02,
         1.04353890e-01,  1.69545747e-02,  9.97742862e-02,
         2.44018957e-01,  4.33962375e-01,  1.22908190e-01,
        -6.83232009e-01,  1.92715988e-01, -5.58317065e-01,
         4.79014277e-01, -1.06621936e-01,  1.98614195e-01,
         1.34860635e-01, -2.68546164e-01,  4.36531544e-01,
         2.53328800e-01, -3.32394205e-02, -7.55754188e-0

In [38]:
df_injury['키워드'].iloc[0]

"['폭행', '회사', '경찰관', '치료', '의사', '형사소송법', '오해', '징계', '사고', '경찰']"

In [39]:
df_injury['키워드'] = df_injury['키워드'].apply(lambda x : x.replace('[','').replace(']','').replace("'","").split(', '))

In [40]:
df_injury['키워드'].iloc[0]

['폭행', '회사', '경찰관', '치료', '의사', '형사소송법', '오해', '징계', '사고', '경찰']

In [41]:
df_injury['클러스터'].value_counts()

클러스터
4    1996
1    1020
3     716
2     457
Name: count, dtype: int64

In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 키워드에 대한 임베딩을 찾기 위한 함수
def get_keyword_embedding(user_keyword, df):
    # 키워드가 포함된 행을 필터링
    matching_rows = df[df['키워드'].apply(lambda x: user_keyword in x if isinstance(x, list) else False)]
    
    if not matching_rows.empty:
        # 첫 번째 매칭된 키워드의 임베딩을 반환
        return matching_rows['키워드임베딩'].iloc[0]
    else:
        return None  # 매칭되는 키워드가 없으면 None 반환

# 사용자 입력
user_keyword = "회사"  # 사용자가 입력한 키워드
user_keyword_embedding = get_keyword_embedding(user_keyword, df_injury)

if user_keyword_embedding is not None:
    # 판례내용 임베딩 유사도 계산 및 사건명 추출
    content_similarity_scores = []

    for idx, row in df_injury.iterrows():
        if row['판례내용이유임베딩'] is not None and not np.isnan(row['판례내용이유임베딩']).any():
            # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
            content_similarity = cosine_similarity(user_keyword_embedding.reshape(1, -1), row['판례내용이유임베딩'].reshape(1, -1)).flatten()[0]
            content_similarity_scores.append((row['사건번호'], content_similarity))

    # 유사도 기준으로 정렬 및 상위 N개 선택
    sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
    top_n_contents = sorted_content_scores[:10]

    # 추천 사건명 출력
    print("사용자 키워드와 관련된 사건명 :")
    for 사건번호, score in top_n_contents:
        사건명 = df_injury.loc[df_injury['사건번호'] == 사건번호, '사건명'].values
        사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
        print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")
else:
    print("해당 키워드에 대한 임베딩이 없습니다.")

사용자 키워드와 관련된 사건명 :
사건번호: 92도737, 사건명: 폭력행위등처벌에관한법률위반,특정경제범죄가중처벌등에관한법률위반,총포·도검·화약류등단속법위반, 유사도: 0.7059
사건번호: 2006노2161, 사건명: 폭력행위등처벌에관한법률위반(집단·흉기등상해), 유사도: 0.7051
사건번호: 85도1717, 사건명: 자동차운전면허취소처분취소, 유사도: 0.7032
사건번호: 2004가합2605, 사건명: 손해배상(기)등, 유사도: 0.6963
사건번호: 89도2260, 사건명: 특정범죄가중처벌등에관한법률위반,강도상해,폭력행위등처벌에관한법률위반, 유사도: 0.6950
사건번호: 83도1258, 사건명: 국회의원선거법위반등피고사건, 유사도: 0.6930
사건번호: 83도3343, 사건명: 강도상해ㆍ특수강도ㆍ특수강도미수ㆍ강도예비ㆍ폭력행위등처벌에관한법률위반, 유사도: 0.6926
사건번호: 85도2514, 사건명: 상해치사,향정신성의약품관리위반, 유사도: 0.6917
사건번호: 83도2407, 사건명: 폭력행위등처벌에관한법률위반, 유사도: 0.6909
사건번호: 2002도6154, 사건명: 상해·금융실명거래및비밀보장에관한법률위반, 유사도: 0.6906


# 데이터베이스에 임베딩된 파일 저장

In [53]:
df_injury_embedding = pd.read_csv('data/df_injury_embedding.csv', index_col=False)
df_injury_embedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4189 non-null   int64 
 1   사건번호           4189 non-null   object
 2   선고일자           4189 non-null   object
 3   판시사항           3417 non-null   object
 4   판결요지           3028 non-null   object
 5   판례내용_상단        4189 non-null   object
 6   판례내용_이유        4184 non-null   object
 7   판례내용_이유_전처리    4189 non-null   object
 8   판례내용_이유_불용어제거  4184 non-null   object
 9   클러스터           4189 non-null   int64 
 10  키워드            4189 non-null   object
 11  사건명            4188 non-null   object
 12  판례내용이유임베딩      4184 non-null   object
 13  키워드임베딩         4189 non-null   object
 14  판결요지임베딩        3028 non-null   object
 15  판시사항임베딩        3417 non-null   object
dtypes: int64(2), object(14)
memory usage: 523.8+ KB


In [54]:
df_injury_embedding.drop(columns=['선고일자','판시사항','판결요지','판례내용_상단','판례내용_이유','판례내용_이유_전처리','판례내용_이유_불용어제거','클러스터','키워드'], inplace=True)
df_injury_embedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   판례일련번호     4189 non-null   int64 
 1   사건번호       4189 non-null   object
 2   사건명        4188 non-null   object
 3   판례내용이유임베딩  4184 non-null   object
 4   키워드임베딩     4189 non-null   object
 5   판결요지임베딩    3028 non-null   object
 6   판시사항임베딩    3417 non-null   object
dtypes: int64(1), object(6)
memory usage: 229.2+ KB


In [55]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

####pandas dataframe에서는 불러올때 타입을 기본str로 불러와서 긴내용의 데이터삽입을 하려면 dtype을 설정해줘야함####
dtype = {
    '사건번호' : TEXT,
    '사건명': TEXT,
    '판례내용임베딩': TEXT,
    '키워드임베딩': TEXT,
    '판결요지임베딩': TEXT,
    '판시사항임베딩': TEXT
}
#판례내용_이유,판례내용_이유(전처리),판례내용_이유(불용어제거),클러스터,키워드

# DB정보
user = user
password = password
host = host
port = port
database = database

# CSV 파일 로드
# df_injury = pd.read_csv('data/상해_본문2.csv')
# df_fraud = pd.read_csv('data/사기_본문2.csv')
# df_inheritance = pd.read_csv('data/상속_본문2.csv')
# df_labor = pd.read_csv('data/근로_본문2.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
# df_injury.to_sql(name='injury_content', con=engine, if_exists='replace', index=False, dtype=dtype)
# df_fraud.to_sql('fraud_content', con=engine, if_exists='replace', index=False, dtype=dtype)
# df_inheritance.to_sql('inheritance_content', con=engine, if_exists='replace', index=False, dtype=dtype)
# df_labor.to_sql('labor_content', con=engine, if_exists='replace', index=False, dtype=dtype)
df_injury_embedding.to_sql('injury_embedding', con=engine, if_exists='replace', index=False, dtype=dtype)

# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")

데이터베이스 저장 완료
